# Part 1 Data Process
_Readme_

The purpose of this notebook is to process the data with different timestamps. Since we have 3 (or more) datasets and each product in these datasets seem to have different timestamps and timelapse, it makes regression analysis very messy. 

## In a word, this notebook merges all the data into one single dense table with consistent timestamp.

In [63]:
import numpy as np
import pandas as pd
import scipy.stats as st
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model as lm
import math
import matplotlib.pyplot as plt
%matplotlib inline
import operator

### Note: I modified the data formats in the xlsx so that they are consistent now
### ECFL stands for "Equity_Credit_FED, LIBOR"

In [64]:
# Read the file
file_com = pd.ExcelFile('rawdata_commodities.xlsx')
file_fx = pd.ExcelFile('Currency.xlsx')
file_ECFL = pd.ExcelFile('ECFL.xlsx')
file_economic = pd.ExcelFile('GDP Inflation Standaradized.xlsx')


In [73]:
# Check the sheet names
[file_com.sheet_names,file_ECFL.sheet_names,file_economic.sheet_names]

[['CO', 'Wheat', 'Gold', 'LinkingMetadata'],
 ['Equity',
  'SHA',
  'TNX',
  'USAgg',
  'GlobalAgg',
  'JP10Y',
  'FED',
  'LIBOR',
  'LinkingMetadata'],
 ['GDP', 'Inflation']]

### 1.1 Data I/O and set dates as index

In [90]:
# Load sheet name into data frames
df_CO = file_com.parse('CO')
df_Wheat = file_com.parse('Wheat')
df_Gold = file_com.parse('Gold')
df_forex = file_fx.parse('Sheet1')
df_GDP = file_economic.parse('GDP')
df_inflation = file_economic.parse('Inflation')

In [91]:
df_CO.columns = ['Date','CO']
df_Wheat.columns = ['Date','Wheat','SMAVG']
df_Gold.columns = ['Date','Gold']
df_GDP.columns = ['Date','GDPGrowth']
df_inflation.columns = ['Date','Inflation']

In [92]:
df_CO.index = df_CO.Date
df_CO.drop('Date',1, inplace=True)
df_Wheat.index = df_Wheat.Date
df_Wheat.drop('Date',1, inplace=True)
df_Gold.index = df_Gold.Date
df_Gold.drop('Date',1, inplace=True)
df_forex.index = df_forex.Date
df_forex.drop('Date', 1, inplace = True)
df_GDP.index = df_GDP.Date
df_GDP.drop('Date',1, inplace=True)
df_inflation.index = df_inflation.Date
df_inflation.drop('Date',1, inplace=True)

In [93]:
file_ECFL.sheet_names

['Equity',
 'SHA',
 'TNX',
 'USAgg',
 'GlobalAgg',
 'JP10Y',
 'FED',
 'LIBOR',
 'LinkingMetadata']

### 1.2 Merge commodities data and forex data 

In [94]:
data = pd.concat([df_CO,df_Wheat,df_Gold,df_forex,df_GDP,df_inflation],join = 'outer',axis = 1)

### 1.3 Include ECFL data using a loop

In [95]:
for sheetName in file_ECFL.sheet_names[0:-1]:
    df = file_ECFL.parse(sheetName)
    df.index = df.Date
    df.drop('Date',1, inplace=True)
    data = pd.concat([data,df],join = 'outer', axis = 1)

# file_additional = pd.ExcelFile('additional data.xlsx')
# df_add = file_additional.parse('Additional Data')
# df_add.columns = ['Date','MSCI EM', 'MSCI EAFE','MSCI World','S&P 500','Russell 1000','Russell 2000','Russell 3000']
# df_add.index = df.Date
# df_add.drop('Date',1, inplace=True)

# data = pd.concat([data,df_add],join = 'outer', axis = 1)

# Sort the data by time
data.sort_index(inplace=True)
    

There are a lot of NAs because on someday we have some data, on others we have other data.
One way to handle it is to use linear interploration

In [96]:
data.interpolate(method='linear', axis=0, limit=None, inplace=True, limit_direction='forward', downcast=None)
data

,CO,Wheat,SMAVG,Gold,USD/CNY,USD/EUR,USD/MYR,USD/AUD,USD/GBP,USD/JPY,...,FTSE,DAX,SHA,TNX,USAgg,GlobalAgg,GlobalAggHY,JP10Y,FED,LIBOR1Y
Date,,,,,,,,,,,,,,,,,,,,,
1914-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Now that the table is dense, let's check what we have

In [97]:
data.columns

Index(['CO', 'Wheat', 'SMAVG', 'Gold', 'USD/CNY', 'USD/EUR', 'USD/MYR',
       'USD/AUD', 'USD/GBP', 'USD/JPY', 'USD/BRL', 'USD/HKD', 'USD/MXN',
       'USD/CAD', 'NZD', 'GDPGrowth', 'Inflation', 'MXEF', 'MXEA ', 'MSCI',
       'SPX', 'RUI', 'RUT', 'RUA', 'NI225', 'SXXP', 'HSI', 'FTSE', 'DAX',
       'SHA', 'TNX', 'USAgg', 'GlobalAgg', 'GlobalAggHY', 'JP10Y', 'FED',
       'LIBOR1Y'],
      dtype='object')

Export to a file so that everyone can use

In [98]:
writer = pd.ExcelWriter('mergedData_v1.xlsx')
data.to_excel(writer,'Sheet1')
writer.save()